<a href="https://colab.research.google.com/github/Ronin1289/Generating-Building-Drawings-Using-Image-Processing/blob/Experiments/EXPERIMENT_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DeepLSD

##Line Detection using Deep Learning

In [ ]:
apt-get update && apt-get install -y build-essential cmake


SyntaxError: invalid syntax (<ipython-input-1-800a7c2a689f>, line 1)

In [ ]:
rm -rf DeepLSD


In [ ]:
!git clone --recurse-submodules https://github.com/cvg/DeepLSD.git
%cd DeepLSDM


Cloning into 'DeepLSD'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 266 (delta 29), reused 33 (delta 22), pack-reused 217 (from 1)
Receiving objects: 100% (266/266), 7.50 MiB | 33.24 MiB/s, done.
Resolving deltas: 100% (116/116), done.
Submodule 'line_refinement/pybind11' (https://github.com/pybind/pybind11) registered for path 'line_refinement/pybind11'
Submodule 'third_party/homography_est' (https://github.com/rpautrat/homography_est.git) registered for path 'third_party/homography_est'
Submodule 'third_party/progressive-x' (https://github.com/danini/progressive-x.git) registered for path 'third_party/progressive-x'
Submodule 'third_party/pytlbd' (https://github.com/iago-suarez/pytlbd.git) registered for path 'third_party/pytlbd'
Submodule 'third_party/pytlsd' (https://github.com/iago-suarez/pytlsd.git) registered for path 'third_party/pytlsd'
Cloning into '/content/DeepLSD/Deep

In [ ]:
!pip install -r requirements.txt


Processing ./third_party/pytlsd
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached brewer2mpl-1.4.1-py2.py3-none-any.whl.metadata (3.7 kB)
  Using cached flow_vis-0.1-py3-none-any.whl.metadata (731 bytes)
  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached jupyterlab-4.3.5-py3-none-any.whl.metadata (16 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl.metadata (4.5 kB)
  Using cached ipykernel-6.29.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached jupyter_lsp-2.2.5-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.15.0-py3-none-any.whl.metadata (8.4 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached comm-0.2.2-py3-none-any.whl.metadata (3.7 kB)
  Using cached jedi-0.19.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached jupyter_client-7.4.9-py3-none-any.whl.metadata (8.5 kB)
  Using cached ju